### 더미 데이터를 활용한 감정 분석 모델링
앞에서 알아본 방법을 활용해 모델을 직접 구현해보자.  
모델 학습에는 keras 모델의 내장 API를 활용할 예정이며, 모델을 구축하는 다양한 방법을 사용해보겠다.  

이번에 구현할 모델은 심층 신경망(Deep Neural Network) 구조를 사용해 앞서 텍스트의 긍정 / 부정을 예측하는 감정 분석(Sentiment Analysis)모델이다.  

In [20]:
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras import layers
import numpy as np

In [27]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

targets = [[1], [0], [1], [1], [0], [1]]
tokenizer = preprocessing.text.Tokenizer() 
tokenizer.fit_on_texts(samples)
labels = np.array(targets)
sequences = tokenizer.texts_to_sequences(samples)
input_sequences = np.array(sequences)
word_index = tokenizer.word_index

In [5]:
batch_size = 2
num_epochs = 100
vocab_size = len(word_index) + 1
emb_size = 128
hidden_dimension = 256
output_dimension = 1 

In [16]:
model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, emb_size, input_length = 4))
# 평균의 경우 하나의 층으로 정의돼 있지 않으므로 람다 층을 활용해 해당 층에 들어오는 입력값들을 평균함
model.add(layers.Lambda(lambda x : tf.reduce_mean(x, axis = 1)))
model.add(layers.Dense(hidden_dimension, activation = 'relu'))
# 최종 출력값의 경우 0과 1 사이의 확률값을 뽑기 위해 활성화 함수로 sigmoid 함수 사용
model.add(layers.Dense(output_dimension, activation = 'sigmoid'))

keras 내장 API를 활용해 모델을 학습하기 위해 모델을 컴파일하는 메서드를 통해 학습 과정 정의.  
Optimizer의 경우 아담(Adam) 최적화 알고리즘을 사용하고,  
학습은 이진 분류 문제이므로 이진 교차 엔트로피 손실 함수(binary cross-entropy)를 사용  
모델의 성능을 측정하기 위한 기준인 평가 지표로 accuracy(정확도) 사용  

In [17]:
model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

컴파일 메서드로 학습 과정을 정의했다면 keras의 내장 API를 통해 학습 진행 

In [28]:
model.fit(input_sequences, labels, epochs = num_epochs, batch_size = batch_size)

Epoch 1/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6954 - accuracy: 0.1667
Epoch 2/100
3/3 [==============================] - 0s 667us/step - loss: 0.6749 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 667us/step - loss: 0.6574 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6408 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 667us/step - loss: 0.6192 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 667us/step - loss: 0.5952 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5686 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 668us/step - loss: 0.5342 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 667us/step - loss: 0.5019 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 1ms/step - loss: 0.4574 - accuracy: 1.0000
Epoch 11/100
3/3 

3/3 [==============================] - 0s 667us/step - loss: 8.1860e-04 - accuracy: 1.0000
Epoch 84/100
3/3 [==============================] - 0s 667us/step - loss: 7.9979e-04 - accuracy: 1.0000
Epoch 85/100
3/3 [==============================] - 0s 1ms/step - loss: 7.7713e-04 - accuracy: 1.0000
Epoch 86/100
3/3 [==============================] - 0s 667us/step - loss: 7.6131e-04 - accuracy: 1.0000
Epoch 87/100
3/3 [==============================] - 0s 667us/step - loss: 7.4419e-04 - accuracy: 1.0000
Epoch 88/100
3/3 [==============================] - 0s 667us/step - loss: 7.2740e-04 - accuracy: 1.0000
Epoch 89/100
3/3 [==============================] - 0s 1ms/step - loss: 7.0850e-04 - accuracy: 1.0000
Epoch 90/100
3/3 [==============================] - 0s 667us/step - loss: 6.9485e-04 - accuracy: 1.0000
Epoch 91/100
3/3 [==============================] - 0s 667us/step - loss: 6.7884e-04 - accuracy: 1.0000
Epoch 92/100
3/3 [==============================] - 0s 667us/step - loss: 6.6418e

fit 메서드 호출 시 학습 경과가 출력되는 것을 확인할 수 있음  
epoch이 경과함에 따라 loss function 값과 accuracy가 늘어가는 것을 확인할 수 있음. 

이제 Funcional API, Subclassing 방법으로 동일한 모델을 구현하고 학습해보자.

- Functional API  
keras의 입력층을 구현한 후 각 값을 다음 layer들을 호출하면서 인자로 넣는 방식으로 구현.  

In [31]:
inputs = layers.Input(shape = (4, ))
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_output = tf.reduce_mean(embed_output, axis = 1)
hidden_layer = layers.Dense(hidden_dimension, activation = 'relu')(pooled_output)
outputs = layers.Dense(output_dimension, activation = 'sigmoid')(hidden_layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)

In [32]:
model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

model.fit(input_sequences, labels, epochs = num_epochs, batch_size = batch_size)

Epoch 1/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6978 - accuracy: 0.1667
Epoch 2/100
3/3 [==============================] - 0s 667us/step - loss: 0.6772 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 667us/step - loss: 0.6606 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6442 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 667us/step - loss: 0.6259 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 667us/step - loss: 0.6026 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 667us/step - loss: 0.5751 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5446 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 667us/step - loss: 0.5069 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 667us/step - loss: 0.4651 - accuracy: 1.0000
Epoch 11/100
3/

3/3 [==============================] - 0s 667us/step - loss: 8.9239e-04 - accuracy: 1.0000
Epoch 84/100
3/3 [==============================] - 0s 668us/step - loss: 8.7411e-04 - accuracy: 1.0000
Epoch 85/100
3/3 [==============================] - 0s 667us/step - loss: 8.5276e-04 - accuracy: 1.0000
Epoch 86/100
3/3 [==============================] - 0s 667us/step - loss: 8.3039e-04 - accuracy: 1.0000
Epoch 87/100
3/3 [==============================] - 0s 1ms/step - loss: 8.1373e-04 - accuracy: 1.0000
Epoch 88/100
3/3 [==============================] - 0s 1ms/step - loss: 7.9528e-04 - accuracy: 1.0000
Epoch 89/100
3/3 [==============================] - 0s 667us/step - loss: 7.7653e-04 - accuracy: 1.0000
Epoch 90/100
3/3 [==============================] - 0s 667us/step - loss: 7.6150e-04 - accuracy: 1.0000
Epoch 91/100
3/3 [==============================] - 0s 667us/step - loss: 7.4577e-04 - accuracy: 1.0000
Epoch 92/100
3/3 [==============================] - 0s 667us/step - loss: 7.2881e

In [33]:
class CustomModel(tf.keras.Model) :
    
    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension) :
        super(CustomModel, self).__init__(name = 'my_model')
        self.embedding = layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = layers.Dense(hidden_dimension, activation = 'relu')
        self.output_layer = layers.Dense(output_dimension, activation = 'sigmoid')
        
    def call(self, inputs) :
        x = self.embedding(inputs)
        x = tf.reduce_mean(x, axis = 1)
        x = self.dense_layer(x)
        x = self.output_layer(x)
        
        return x

In [34]:
model = CustomModel(vocab_size = vocab_size,
                   embed_dimension = emb_size,
                   hidden_dimension = hidden_dimension,
                   output_dimension = output_dimension)

model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

model.fit(input_sequences, labels, epochs = num_epochs, batch_size = batch_size)

Epoch 1/100
3/3 [==============================] - 0s 667us/step - loss: 0.6951 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 670us/step - loss: 0.6761 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 998us/step - loss: 0.6600 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6448 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 667us/step - loss: 0.6259 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 666us/step - loss: 0.6033 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 667us/step - loss: 0.5779 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 668us/step - loss: 0.5484 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 667us/step - loss: 0.5145 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 667us/step - loss: 0.4729 - accuracy: 1.0000
Epoch 11/10

3/3 [==============================] - 0s 667us/step - loss: 8.7825e-04 - accuracy: 1.0000
Epoch 83/100
3/3 [==============================] - 0s 333us/step - loss: 8.5273e-04 - accuracy: 1.0000
Epoch 84/100
3/3 [==============================] - 0s 1ms/step - loss: 8.3435e-04 - accuracy: 1.0000
Epoch 85/100
3/3 [==============================] - 0s 667us/step - loss: 8.1530e-04 - accuracy: 1.0000
Epoch 86/100
3/3 [==============================] - 0s 667us/step - loss: 7.9654e-04 - accuracy: 1.0000
Epoch 87/100
3/3 [==============================] - 0s 667us/step - loss: 7.7435e-04 - accuracy: 1.0000
Epoch 88/100
3/3 [==============================] - 0s 667us/step - loss: 7.5777e-04 - accuracy: 1.0000
Epoch 89/100
3/3 [==============================] - 0s 667us/step - loss: 7.3937e-04 - accuracy: 1.0000
Epoch 90/100
3/3 [==============================] - 0s 667us/step - loss: 7.2196e-04 - accuracy: 1.0000
Epoch 91/100
3/3 [==============================] - 0s 668us/step - loss: 7.070